In [ ]:
pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.3 MB/s 
     |████████████████████████████████| 232 kB 32.8 MB/s 
     |████████████████████████████████| 51 kB 8.3 MB/s 
     |████████████████████████████████| 4.4 MB 14.4 MB/s 
     |████████████████████████████████| 1.2 MB 54.0 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 596 kB 45.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.8 MB/s 
  Created wheel for keybert: filename=keybert-0.5.1-py3-none-any.whl size=21332 sha256=f53f092740c317bfa29b9299455796cdc63b26aecdefa8aa988a0ce6a9d5e8f6
  Stored in directory: /root/.cache/pip/wheels/8e/95/c5/f5ceed2a9f9e80bc1a706a10a6fb03d726df7a3dd11800a58b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6376c49bc6ad1ebc3642c1427a7238969fd49f8457fbd1034abaf10147ad525f
 

In [ ]:
import pandas as pd
import string
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from keybert import KeyBERT
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from transformers import BertTokenizer, TFBertModel, TFAutoModel,AutoTokenizer
import matplotlib.pyplot as plt

# nltk.download()
nltk.download('omw-1.4')
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data = pd.read_csv("RUG.csv",encoding="ISO-8859-1")
pd.set_option('display.max_colwidth', -1)
data= data [['Outcome', 'Description']][1:]
# data.head()

**function to clear data from punctuations**

In [ ]:
def remove_punctuation(text):
    # punctuationfree="".join([i for i in text if i not in string.punctuation])
  punctuationfree = ""
  if(text is np.nan):
    return punctuationfree
  for i in range(len(text)):
    if(text[i] not in string.punctuation):
      punctuationfree = punctuationfree + text[i]
  return punctuationfree

**cleaning the data from punctuations and lowerize**

In [ ]:
data['clean_outcome']= data['Outcome'].apply(lambda x:remove_punctuation(x))
data['clean_description']= data['Description'].apply(lambda x:remove_punctuation(x))

data['clean_outcome']= data['clean_outcome'].apply(lambda x: x.lower())
data['clean_description']= data['clean_description'].apply(lambda x: x.lower())

# data.head()

**tokenize data**

In [ ]:
def tokenization(text):
  PATTERN = r"\w+"
  tokens = re.findall(PATTERN, text)
  return tokens
data['tokenied_outcome']= data['clean_outcome'].apply(lambda x: tokenization(x))
data['tokenied_description']= data['clean_description'].apply(lambda x: tokenization(x))
# data.head()

**removing stopwords**

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

data['tokenied_outcome']= data['tokenied_outcome'].apply(lambda x: remove_stopwords(x))
data['tokenied_description']= data['tokenied_description'].apply(lambda x: remove_stopwords(x))

**stemming data**

In [ ]:
porter_stemmer = PorterStemmer()
def stemming(text):
  stem_text = [porter_stemmer.stem(word) for word in text]
  return stem_text
data['stemmed_outcome']=data['tokenied_outcome'].apply(lambda x: stemming(x))
data['stemmed_description']=data['tokenied_description'].apply(lambda x: stemming(x))

**lemmatize data**

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text

data['lemmatized_outcome']=data['tokenied_outcome'].apply(lambda x: lemmatizer(x))
data['lemmatized_description']=data['tokenied_description'].apply(lambda x: lemmatizer(x))
data.head()

,Outcome,Description,clean_outcome,clean_description,tokenied_outcome,tokenied_description,stemmed_outcome,stemmed_description,lemmatized_outcome,lemmatized_description
1,"Aims:- To obtain knowledge of and insight into fundamental (stem) cell biology processes and principles;- To obtain knowledge in fundamental processes in (epi)genetics;- To understand the relation between stem cells, disease and therapy.Skills:- To be able to discuss basic stem cell biology and to convey this information to a broad audience;- To be able to understand and identify possible therapeutic applications and to convey this information to a broad audience and to the media;- To be able to identify novel trends and future perspectives for stem cell biology but also to understand limitations;- To be able to discuss ethical issues of stem cell biology.","Stem cell biology is a rapidly developing research field that has large implications for our understanding of human genetics, human disease and ageing. The technology applied in stem cell research is very complex and the constant development of knowledge is fast. This makes the supply of information to a broad audience and to the media a challenge.In a series of lectures we will explore all relevant issues on technology, practical applications and future perspectives of stem cell biology.The course will be supported by a team of stem cell specialists that will provide lectures and reading material, the course also includes a stem cell laboratory tour.",aims to obtain knowledge of and insight into fundamental stem cell biology processes and principles to obtain knowledge in fundamental processes in epigenetics to understand the relation between stem cells disease and therapyskills to be able to discuss basic stem cell biology and to convey this information to a broad audience to be able to understand and identify possible therapeutic applications and to convey this information to a broad audience and to the media to be able to identify novel trends and future perspectives for stem cell biology but also to understand limitations to be able to discuss ethical issues of stem cell biology,stem cell biology is a rapidly developing research field that has large implications for our understanding of human genetics human disease and ageing the technology applied in stem cell research is very complex and the constant development of knowledge is fast this makes the supply of information to a broad audience and to the media a challengein a series of lectures we will explore all relevant issues on technology practical applications and future perspectives of stem cell biologythe course will be supported by a team of stem cell specialists that will provide lectures and reading material the course also includes a stem cell laboratory tour,"[aims, obtain, knowledge, insight, fundamental, stem, cell, biology, processes, principles, obtain, knowledge, fundamental, processes, epigenetics, understand, relation, stem, cells, disease, therapyskills, able, discuss, basic, stem, cell, biology, convey, information, broad, audience, able, understand, identify, possible, therapeutic, applications, convey, information, broad, audience, media, able, identify, novel, trends, future, perspectives, stem, cell, biology, also, understand, limitations, able, discuss, ethical, issues, stem, cell, biology]","[stem, cell, biology, rapidly, developing, research, field, large, implications, understanding, human, genetics, human, disease, ageing, technology, applied, stem, cell, research, complex, constant, development, knowledge, fast, makes, supply, information, broad, audience, media, challengein, series, lectures, explore, relevant, issues, technology, practical, applications, future, perspectives, stem, cell, biologythe, course, supported, team, stem, cell, specialists, provide, lectures, reading, material, course, also, includes, stem, cell, laboratory, tour]","[aim, obtain, knowledg, insight, fundament, stem, cell, biolog, process, princi

**Keyword Extraction**

In [ ]:
def listToString(s):
    str1 = " "
    # print(str1.join(s))
    return (str1.join(s))

data['lemmatized_outcome_s']=data['lemmatized_outcome'].apply(lambda x: listToString(x))
data['lemmatized_description_s']=data['lemmatized_description'].apply(lambda x: listToString(x))
data.head()
# s = ['Geeks', 'for', 'Geeks', 'for', 'Geeks', 'for', 'Geeks', 'for', 'Geeks']
# data['lemmatized_outcome'][1]
# data['lemmatized_outcome'][0:5]

,Outcome,Description,clean_outcome,clean_description,tokenied_outcome,tokenied_description,stemmed_outcome,stemmed_description,lemmatized_outcome,lemmatized_description,lemmatized_outcome_s,lemmatized_description_s
1,"Aims:- To obtain knowledge of and insight into fundamental (stem) cell biology processes and principles;- To obtain knowledge in fundamental processes in (epi)genetics;- To understand the relation between stem cells, disease and therapy.Skills:- To be able to discuss basic stem cell biology and to convey this information to a broad audience;- To be able to understand and identify possible therapeutic applications and to convey this information to a broad audience and to the media;- To be able to identify novel trends and future perspectives for stem cell biology but also to understand limitations;- To be able to discuss ethical issues of stem cell biology.","Stem cell biology is a rapidly developing research field that has large implications for our understanding of human genetics, human disease and ageing. The technology applied in stem cell research is very complex and the constant development of knowledge is fast. This makes the supply of information to a broad audience and to the media a challenge.In a series of lectures we will explore all relevant issues on technology, practical applications and future perspectives of stem cell biology.The course will be supported by a team of stem cell specialists that will provide lectures and reading material, the course also includes a stem cell laboratory tour.",aims to obtain knowledge of and insight into fundamental stem cell biology processes and principles to obtain knowledge in fundamental processes in epigenetics to understand the relation between stem cells disease and therapyskills to be able to discuss basic stem cell biology and to convey this information to a broad audience to be able to understand and identify possible therapeutic applications and to convey this information to a broad audience and to the media to be able to identify novel trends and future perspectives for stem cell biology but also to understand limitations to be able to discuss ethical issues of stem cell biology,stem cell biology is a rapidly developing research field that has large implications for our understanding of human genetics human disease and ageing the technology applied in stem cell research is very complex and the constant development of knowledge is fast this makes the supply of information to a broad audience and to the media a challengein a series of lectures we will explore all relevant issues on technology practical applications and future perspectives of stem cell biologythe course will be supported by a team of stem cell specialists that will provide lectures and reading material the course also includes a stem cell laboratory tour,"[aims, obtain, knowledge, insight, fundamental, stem, cell, biology, processes, principles, obtain, knowledge, fundamental, processes, epigenetics, understand, relation, stem, cells, disease, therapyskills, able, discuss, basic, stem, cell, biology, convey, information, broad, audience, able, understand, identify, possible, therapeutic, applications, convey, information, broad, audience, media, able, identify, novel, trends, future, perspectives, stem, cell, biology, also, understand, limitations, able, discuss, ethical, issues, stem, cell, biology]","[stem, cell, biology, rapidly, developing, research, field, large, implications, understanding, human, genetics, human, disease, ageing, technology, applied, stem, cell, research, complex, constant, development, knowledge, fast, makes, supply, information, broad, audience, media, challengein, series, lectures, explore, relevant, issues, technology, practical, applications, future, perspectives, stem, cell, biologythe, course, supported, team, stem, cell, specialists, provide, lectures, reading, material, course, also, includes, stem, cell, laboratory, tour]","[aim, obtain, knowledg, insight, 

In [ ]:
kw_model = KeyBERT()
# keywords = kw_model.extract_keywords(data['lemmatized_outcome'][2], highlight=True)

keywords = []
for item in data['lemmatized_outcome_s']:
  keywords.append(kw_model.extract_keywords(item))

data['keywords'] = keywords

keywords = []
for item in data['lemmatized_description_s']:
  keywords.append(kw_model.extract_keywords(item))

# print(data['keywords'][0:5])
for i in range(1, len(data['keywords'])):
  data['keywords'][i] += keywords[i]

# print(data['keywords'][0:5])
data.head()
# print(len(keywords))

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

,Outcome,Description,clean_outcome,clean_description,tokenied_outcome,tokenied_description,stemmed_outcome,stemmed_description,lemmatized_outcome,lemmatized_description,lemmatized_outcome_s,lemmatized_description_s,keywords
1,"Aims:- To obtain knowledge of and insight into fundamental (stem) cell biology processes and principles;- To obtain knowledge in fundamental processes in (epi)genetics;- To understand the relation between stem cells, disease and therapy.Skills:- To be able to discuss basic stem cell biology and to convey this information to a broad audience;- To be able to understand and identify possible therapeutic applications and to convey this information to a broad audience and to the media;- To be able to identify novel trends and future perspectives for stem cell biology but also to understand limitations;- To be able to discuss ethical issues of stem cell biology.","Stem cell biology is a rapidly developing research field that has large implications for our understanding of human genetics, human disease and ageing. The technology applied in stem cell research is very complex and the constant development of knowledge is fast. This makes the supply of information to a broad audience and to the media a challenge.In a series of lectures we will explore all relevant issues on technology, practical applications and future perspectives of stem cell biology.The course will be supported by a team of stem cell specialists that will provide lectures and reading material, the course also includes a stem cell laboratory tour.",aims to obtain knowledge of and insight into fundamental stem cell biology processes and principles to obtain knowledge in fundamental processes in epigenetics to understand the relation between stem cells disease and therapyskills to be able to discuss basic stem cell biology and to convey this information to a broad audience to be able to understand and identify possible therapeutic applications and to convey this information to a broad audience and to the media to be able to identify novel trends and future perspectives for stem cell biology but also to understand limitations to be able to discuss ethical issues of stem cell biology,stem cell biology is a rapidly developing research field that has large implications for our understanding of human genetics human disease and ageing the technology applied in stem cell research is very complex and the constant development of knowledge is fast this makes the supply of information to a broad audience and to the media a challengein a series of lectures we will explore all relevant issues on technology practical applications and future perspectives of stem cell biologythe course will be supported by a team of stem cell specialists that will provide lectures and reading material the course also includes a stem cell laboratory tour,"[aims, obtain, knowledge, insight, fundamental, stem, cell, biology, processes, principles, obtain, knowledge, fundamental, processes, epigenetics, understand, relation, stem, cells, disease, therapyskills, able, discuss, basic, stem, cell, biology, convey, information, broad, audience, able, understand, identify, possible, therapeutic, applications, convey, information, broad, audience, media, able, identify, novel, trends, future, perspectives, stem, cell, biology, also, understand, limitations, able, discuss, ethical, issues, stem, cell, biology]","[stem, cell, biology, rapidly, developing, research, field, large, implications, understanding, human, genetics, human, disease, ageing, technology, applied, stem, cell, research, complex, constant, development, knowledge, fast, makes, supply, information, broad, audience, media, challengein, series, lectures, explore, relevant, issues, technology, practical, applications, future, perspectives, stem, cell, biologythe, course, supported, team, stem, cell, specialists, provide, lectures, reading, material, course, also, includes, stem, cell, laboratory, tour]","[aim, obtain, knowledg, 

In [ ]:
dataset = []
for course in data['keywords']:
  keys = []
  for key in course:
    keys.append(key[0])
    # print(key[0])
  dataset.append(keys)
dataset

[['stem',
  'biology',
  'knowledge',
  'cell',
  'audience',
  'studenten',
  'computerpracticum',
  'practicum',
  'onderzoeksdeelnamepracticum',
  'experimenten'],
 ['psychologische',
  'psychologisch',
  'experimenten',
  'onderzoekspractica',
  'onderzoekspraktijk',
  'sexuality',
  'gender',
  'pedagogical',
  'educational',
  'study'],
 ['gender',
  'sexuality',
  'sociocontextual',
  'sexual',
  'education',
  'psychologische',
  'psychologie',
  'studenten',
  'wetenschappelijke',
  'study'],
 ['onderzoeksmethoden',
  'onderzoeksmethodenâ',
  'wetenschappelijke',
  'nietwetenschappelijke',
  'onderzoeksconstructies',
  'aanbestedingsrechtelijke',
  'europeesrechtelijke',
  'aanbestedingsrecht',
  'privaatrecht',
  'mededingingsrecht'],
 ['aanbestedingsrechtelijke',
  'aanbestedingsrecht',
  'studenten',
  'nederlandse',
  'het',
  'extractive',
  'groningen',
  'gas',
  'extraction',
  'extract'],
 ['insight',
  'activism',
  'research',
  'earthquake',
  'politics',
  'studen

**Generating Frequent Itemsets**

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,3d,aanbestedingsrecht,aanbestedingsrechtelijke,ability,abm,abstract,abstraction,academia,academic,accelerator,...,zalworden,zeggen,zelfstandig,zelfstudie,zelfstudieopdrachten,zij,zijn,zijnhaar,zoning,zooarchaeology
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
246,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
247,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
248,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


**association rules**

In [ ]:
# apriori(df, min_support=0.01)
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets
# frequent_itemsets[ (frequent_itemsets['length'] == 3)]

,support,itemsets,length
0,0.104,(academic),1
1,0.012,(accountancy),1
2,0.024,(accounting),1
3,0.012,(activism),1
4,0.028,(advanced),1
...,...,...,...
359,0.012,"(develop, student, institute, research)",4
360,0.012,"(develop, student, proposal, research)",4
361,0.012,"(student, proposal, institute, research)",4
362,0.012,"(college, design, fse, student, academic)",5


**Vectorize**

In [ ]:
model_name ='joeddav/xlm-roberta-large-xnli'
tokenizer = AutoTokenizer.from_pretrained(model_name)


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
def encode_sentence(s):
    tokens=[]
    tokens.append('[CLS]')
    tokens+=list(tokenizer.tokenize(s))
#     print(tokens)
    return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
main_data=data['lemmatized_outcome_s']
main_data=pd.DataFrame(main_data)
main_data.head()
encode_sentence(main_data['lemmatized_outcome_s'][1])

[3,
 10,
 464,
 113054,
 51359,
 149201,
 20531,
 36418,
 38750,
 333,
 25443,
 9433,
 24702,
 133,
 113054,
 51359,
 20531,
 9433,
 25277,
 15292,
 41637,
 28219,
 41911,
 36418,
 38750,
 70997,
 6,
 106864,
 1336,
 42458,
 19048,
 17116,
 223,
 62822,
 36418,
 38750,
 333,
 25443,
 158,
 67709,
 4677,
 134744,
 95771,
 19048,
 28219,
 135812,
 7722,
 6,
 126472,
 1771,
 38415,
 158,
 67709,
 4677,
 134744,
 95771,
 57646,
 19048,
 135812,
 21261,
 12768,
 22690,
 80280,
 36418,
 38750,
 333,
 25443,
 2843,
 28219,
 205969,
 19048,
 17116,
 223,
 82,
 127,
 21533,
 31089,
 36418,
 38750,
 333,
 25443]

In [ ]:
import sklearn

lemSentences = data['lemmatized_outcome']
stemSentences = data['stemmed_outcome']
stemLemSentences = data['lemmatized_outcome_s']
lemSentences = lemSentences.apply(lambda x: ' '.join(x))
stemSentences = stemSentences.apply(lambda x:' '.join(x))
stemLemSentences = stemLemSentences.apply(lambda x:' '.join(x))

In [ ]:
!pip install keybert
!pip install keybert[flair]
!pip install keybert[gensim]
!pip install keybert[spacy]
!pip install keybert[use]
from keybert import KeyBERT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 776.7 MB 4.4 kB/s 
     |████████████████████████████████| 1.3 MB 58.9 MB/s 
     |████████████████████████████████| 448 kB 72.4 MB/s 
     |████████████████████████████████| 981 kB 56.7 MB/s 
     |████████████████████████████████| 46 kB 5.4 MB/s 
     |████████████████████████████████| 19.7 MB 104.0 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 788 kB 75.1 MB/s 
     |████████████████████████████████| 1.1 MB 65.7 MB/s 
     |████████████████████████████████| 880 kB 68.6 MB/s 
     |████████████████████████████████| 2.9 MB 56.6 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 79 kB 9.0 MB/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model2 = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
# lemembeddings = model.encode(lemSentences)
print(data['lemmatized_outcome'])

1      [aim, obtain, knowledge, insight, fundamental, stem, cell, biology, process, principle, obtain, knowledge, fundamental, process, epigenetics, understand, relation, stem, cell, disease, therapyskills, able, discus, basic, stem, cell, biology, convey, information, broad, audience, able, understand, identify, possible, therapeutic, application, convey, information, broad, audience, medium, able, identify, novel, trend, future, perspective, stem, cell, biology, also, understand, limitation, able, discus, ethical, issue, stem, cell, biology]                                                                                                                                                                                                                                                                                                                                                                                                                                   
2      [na, het, voltooien, van